In [1]:
from mycnn import VGG16
from mycnn import utils
import tensorflow as tf
import numpy as np

In [2]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(224,224),
    batch_size=20,
    label_mode="categorical",
    validation_split=0.2,
    subset="training",
    seed=10
)
train_dataset.map(lambda x, y: (x/255., y))


valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(224,224),
    batch_size=20,
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=10
)
valid_dataset.map(lambda x, y: (x/255., y))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


<MapDataset shapes: ((None, 224, 224, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [3]:
vgg16 = VGG16(classes_num=2)
vgg16.summary()

Model: "VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 224, 224, 64)      256       
_________________________________________________________________
block1_relu1 (ReLU)          (None, 224, 224, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 224, 224, 64)      256       
_________________________________________________________________
block1_relu2 (ReLU)          (None, 224, 224, 64)      0     

In [ ]:
vgg16.setup_training(
    'log_vgg16',
    epochs=20,
    batch_size=20,  # batch size depend on ImageGenerator
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
vgg16.add_callback(
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.8, patience=10,
        min_lr=0.00001,
        verbose=1
    )
)

In [ ]:
vgg16.train_dataset(train_dataset, valid_dataset)

In [ ]:
vgg16.show_history(["loss", "accuracy"])

In [ ]:
vgg16.eval_dataset(valid_dataset)

In [ ]:
pred_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(227,227),
    batch_size=1,
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=10
)

pr_score = vgg16.pred_dataset(pred_dataset)
pr_label = pr_score.argmax(axis=-1)

y_test = []
for d in pred_dataset:
    y_test.append(d[1].numpy()[0])
y_test = np.array(y_test)
gt_label = y_test.argmax(axis=-1)

target_names = ["Cats", "Dogs"]

report = utils.export_classification_report(
    gt_label, pr_label, pr_score,
    target_names=target_names,
    logpath=vgg16.logdir
)

cm = report["confusion_matrix"]
cm_precision = cm/cm.sum(axis=0)
cm_recall = cm/cm.sum(axis=1)
utils.plot_confusion_matrix(cm_recall, target_names, vgg16.logdir, title='Confusion Matrix (recall)')
utils.plot_confusion_matrix(cm_precision, target_names, vgg16.logdir, title='Confusion Matrix (precision)')